In [4]:
from bs4 import BeautifulSoup
import requests
import re

def get_soup(url):

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:57.0) Gecko/20100101 Firefox/57.0'}

    r = s.get(url, headers=headers)

    if r.status_code != 200:
        print('status code:', r.status_code)
    else:
        return BeautifulSoup(r.text, 'html.parser')

def parse(url, response):

    if not response:
        print('no response:', url)
        return

    # get number of reviews
    num_reviews = response.find('div', class_='pageNumbers')
    if num_reviews != None :
        num_reviews = num_reviews.find('span', class_='last').text
        num_reviews = int(num_reviews)
        num_reviews *= 5
    else:
        num_reviews = 5
    print('num_reviews:', num_reviews, type(num_reviews))

    # create template for urls to pages with reviews
    url = url.replace('.html', '-or{}.html')
    print('template:', url)

    # load pages with reviews
    for offset in range(0, num_reviews, 5):
        print('url:', url.format(offset))
        url_ = url.format(offset)
        parse_reviews(url_, get_soup(url_))
#         return # for test only - to stop after first page

def review_body(url, response):
    print('review_url:', url)

    if not response:
        print('no response:', url)
        return
    review_body = response.find('p', class_='partial_entry').text
    return review_body

def parse_reviews(url, response):
    print('review:', url)

    if not response:
        print('no response:', url)
        return

    # get every review
    for idx, review in enumerate(response.find_all('div', class_='review-container')):
        try:
            item = {
                'hotel_name': response.find('h1', class_='heading_title').text,
                'hotel_locality': response.find('span', class_='locality').text,
                'hotel_rating': response.find('span', class_='header_rating').div.div.span['alt'],
                'review_url': 'https://www.tripadvisor.com.au' + review.find('div', class_='quote').a['href'],
                'review_title': review.find('span', class_='noQuotes').text,
                'review_date': review.find('span', class_='relativeDate')['title'],
                'num_reviews_reviewer': review.find('span', class_='badgetext').text,
                'reviewer_name': review.find('span', class_='scrname').text,
                'user_location': review.find('span', class_='userLocation').text,
                'bubble_rating': review.select_one('div.reviewItemInline span.ui_bubble_rating')['class'][1][7:],
            }

            nurl = item['review_url']

            item['review_body'] = review_body(nurl,get_soup(nurl))
            results.append(item) # <--- add to global list
        

            for key,val in item.items():
                print(key, ':', val)
            print('----')
            #return # for test only - to stop after first review
            
        except:
            pass




# --- main ---

s = requests.Session()

start_urls = [
'https://www.tripadvisor.com/Hotel_Review-g293978-d506758-Reviews-Shepherd_Hotel-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d9737305-Reviews-Caritas_Betharram_Center-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d3577109-Reviews-Golden_Park_Resort-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d5244991-Reviews-Grand_Park_Hotel_Bethlehem-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d12324965-Reviews-The_Diamond_Hotel-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d506772-Reviews-Paradise_Hotel_Bethlehem-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d2218659-Reviews-Shepherd_Plaza_Hotel-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g2059914-d1783951-Reviews-Angel_Hotel-Beit_Jala_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d1174151-Reviews-Murad_Tourist_Resort-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d7347638-Reviews-Grand_Hotel_Bethlehem-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d2277541-Reviews-Nativity_Bells_Hotel-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d1794304-Reviews-Bethlehem_Star_Hotel-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d8812735-Reviews-Orient_Palace_Hotel-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g667390-d8873413-Reviews-Saleen_Afandi_Hotel-Nablus_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g663088-d8307376-Reviews-Blue_Beach_Resort-Gaza_City_Gaza.html',
'https://www.tripadvisor.com/Hotel_Review-g667390-d4476290-Reviews-Hotel_Al_Qasr-Nablus_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g663088-d1755125-Reviews-Commodore_Hotel_Gaza-Gaza_City_Gaza.html',
'https://www.tripadvisor.com/Hotel_Review-g663088-d3204871-Reviews-ArcMed_Hotels_Al_Mashtal_Gaza-Gaza_City_Gaza.html',
'https://www.tripadvisor.com/Hotel_Review-g663088-d594409-Reviews-Al_Deira-Gaza_City_Gaza.html',
'https://www.tripadvisor.com/Hotel_Review-g667395-d9708286-Reviews-Haddad_Tourism_Village-Jenin_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g293978-d2025190-Reviews-Alexander_Hotel-Bethlehem_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g667136-d7383224-Reviews-Casablanca_Hotel_Ramallah-Ramallah_Binyamin_Region_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g667390-d11019420-Reviews-Royal_Suites-Nablus_West_Bank.html',
'https://www.tripadvisor.com/Hotel_Review-g663088-d1205501-Reviews-Grand_Palace_Hotel-Gaza_City_Gaza.html',
'https://www.tripadvisor.com/Hotel_Review-g667136-d1918230-Reviews-Grand_Park_Hotel-Ramallah_Binyamin_Region_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d6615598-Reviews-Talitha_Kumi_Guest_House-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d4469435-Reviews-Ararat_Hotel-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d325647-Reviews-Jacir_Palace_Hotel_Bethlehem-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d619513-Reviews-Bethlehem_Hotel-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667136-d1958679-Reviews-Caesar_Hotel-Ramallah_Binyamin_Region_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667390-d10169216-Reviews-Hotel_Crystal_Motel-Nablus_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g12596101-d7724927-Reviews-Taybeh_Golden_Hotel-Taybeh_Binyamin_Region_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d2099106-Reviews-Holy_Family_Hotel-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d3291336-Reviews-Manger_Square_Hotel-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667136-d1572723-Reviews-Royal_Court_Hotel-Ramallah_Binyamin_Region_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667136-d1011353-Reviews-City_Inn_Palace_Hotel-Ramallah_Binyamin_Region_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d12712282-Reviews-Innova8ion-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667136-d12982352-Reviews-Millennium_Palestine_Ramallah-Ramallah_Binyamin_Region_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d1205571-Reviews-Sancta_Maria_Hotel-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667394-d10060492-Reviews-Queen_Plaza_Hotel-Hebron_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g297750-d308910-Reviews-Oasis_Hotel_Jericho-Jericho_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g297750-d657651-Reviews-Jericho_Resort_Village-Jericho_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d4747825-Reviews-Nativity_Hotel-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g663088-d7021114-Reviews-Roots_Hotel-Gaza_City_Gaza.html ',
'https://www.tripadvisor.com/Hotel_Review-g1902315-d2351635-Reviews-Shepherds_House-Beit_Sahour_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g1902315-d11866956-Reviews-Eman_Regency_Hotel-Beit_Sahour_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667136-d2295019-Reviews-Mirador_Hotel_Restaurant-Ramallah_Binyamin_Region_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d3399009-Reviews-Pilgrim_Residence-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g663088-d1974787-Reviews-Al_Mathaf_Recreational_and_Cultural_house-Gaza_City_Gaza.html ',
'https://www.tripadvisor.com/Hotel_Review-g293978-d1911175-Reviews-Sahara_Hotel-Bethlehem_West_Bank.html ',
'https://www.tripadvisor.com/Hotel_Review-g667136-d6416417-Reviews-Palestine_Plaza_Hotel-Ramallah_Binyamin_Region_West_Bank.html ',    

]

print('start_urls: ',len(start_urls))

results = [] # <--- global list for items


for url in start_urls:
    parse(url, get_soup(url))

import pandas as pd

df = pd.DataFrame(results) # <--- convert list to DataFrame
df.to_csv('output.csv')

start_urls:  51
num_reviews: 35 <class 'int'>
template: https://www.tripadvisor.com/Hotel_Review-g293978-d506758-Reviews-Shepherd_Hotel-Bethlehem_West_Bank-or{}.html
url: https://www.tripadvisor.com/Hotel_Review-g293978-d506758-Reviews-Shepherd_Hotel-Bethlehem_West_Bank-or0.html
review: https://www.tripadvisor.com/Hotel_Review-g293978-d506758-Reviews-Shepherd_Hotel-Bethlehem_West_Bank-or0.html
review_url: https://www.tripadvisor.com.au/ShowUserReviews-g293978-d506758-r551774454-Shepherd_Hotel-Bethlehem_West_Bank.html#CHECK_RATES_CONT
hotel_name : Shepherd Hotel
hotel_locality : Bethlehem, 
hotel_rating : 4 of 5 bubbles
review_url : https://www.tripadvisor.com.au/ShowUserReviews-g293978-d506758-r551774454-Shepherd_Hotel-Bethlehem_West_Bank.html#CHECK_RATES_CONT
review_title : Scam Laundry Service
review_date : January 5, 2018
num_reviews_reviewer : 9
reviewer_name : Traveler_10201020
user_location : Vienna, Austria
bubble_rating : 10
review_body : The hotel offers a scam laundry service